# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('zillow_data.csv')

In [3]:
data.head()
# RegionName is the zipcode

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 30.6+ MB


In [35]:
chicago.columns

Index(['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName',
       'SizeRank', '1996-04', '1996-05', '1996-06',
       ...
       '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12',
       '2018-01', '2018-02', '2018-03', '2018-04'],
      dtype='object', length=272)

In [42]:
chicago = data.loc[data['Metro'] == 'Chicago']
chicago.drop(['RegionID', 'City', 'State', 'Metro', 'CountyName','SizeRank'], axis=1).set_index(['RegionName']).T

RegionName,60657,60614,60640,60647,60618,60613,60629,60625,60619,60610,...,60043,47963,60556,60444,60407,60511,60150,60180,47948,60512
1996-04,334200.0,498100.0,216500.0,122700.0,142600.0,297900.0,93400.0,148900.0,76400.0,435200.0,...,661700.0,NaN,135100.0,90900.0,73500.0,167200.0,133000.0,162900.0,NaN,146800.0
1996-05,335400.0,500900.0,216700.0,122800.0,143100.0,300400.0,94100.0,149300.0,77000.0,435200.0,...,662900.0,NaN,134800.0,91300.0,73800.0,166600.0,132800.0,162400.0,NaN,146400.0
1996-06,336500.0,503100.0,216900.0,122800.0,143400.0,302600.0,94600.0,149600.0,77400.0,435100.0,...,663800.0,NaN,134500.0,91700.0,74100.0,165900.0,132500.0,161900.0,NaN,146000.0
1996-07,337600.0,504600.0,217000.0,122700.0,143300.0,304700.0,94800.0,149700.0,77700.0,434600.0,...,664400.0,NaN,134100.0,92100.0,74400.0,165100.0,132200.0,161300.0,NaN,145600.0
1996-08,338500.0,505500.0,217100.0,122400.0,142900.0,306600.0,94700.0,149700.0,77700.0,433900.0,...,664800.0,NaN,133700.0,92600.0,74700.0,164300.0,131900.0,160800.0,NaN,145200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12,1018700.0,1299000.0,777900.0,470600.0,457900.0,1041300.0,162600.0,463100.0,127800.0,1045700.0,...,1348600.0,88100.0,146900.0,156600.0,124400.0,233600.0,133000.0,226400.0,82200.0,209900.0
2018-01,1024400.0,1302700.0,778500.0,474500.0,457400.0,1051300.0,163500.0,464100.0,127100.0,1050800.0,...,1357400.0,86800.0,146500.0,156900.0,124100.0,235200.0,134200.0,229300.0,80300.0,211700.0
2018-02,1030700.0,1306400.0,780500.0,475100.0,459000.0,1059300.0,164000.0,464200.0,126800.0,1050400.0,...,1362600.0,86100.0,146800.0,157000.0,123300.0,236500.0,135700.0,231500.0,79600.0,213200.0
2018-03,1033800.0,1308500.0,782800.0,472600.0,462500.0,1060200.0,164000.0,463200.0,128000.0,1045000.0,...,1363500.0,86200.0,149300.0,156600.0,121800.0,237300.0,138700.0,234500.0,81300.0,214600.0


# Step 2: Data Preprocessing

In [23]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[7:], format='%Y-%m')

In [24]:
get_datetimes(chicago)

DatetimeIndex(['1996-04-01', '1996-05-01', '1996-06-01', '1996-07-01',
               '1996-08-01', '1996-09-01', '1996-10-01', '1996-11-01',
               '1996-12-01', '1997-01-01',
               ...
               '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01',
               '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
               '2018-03-01', '2018-04-01'],
              dtype='datetime64[ns]', length=265, freq=None)

In [27]:
type(chicago.columns[8])

str

# Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

# Step 4: Reshape from Wide to Long Format

In [ ]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

# Step 5: ARIMA Modeling

# Step 6: Interpreting Results